In [13]:
# !pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu



from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [1]:
# !pip uninstall -y keras tensorflow tensorflow-keras tf-keras transformers
# !pip install tensorflow==2.18.0 keras==3.8.0 transformers==4.48.3

In [2]:
import tensorflow as tf
print(tf.__version__, tf.keras.__version__)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/stygianphantom/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Us

2.19.0 3.9.0


In [3]:
# !pip install tensorflow==2.18.0 transformers==4.48.3

In [ ]:
# !pip install transformers datasets accelerate # If not installed
# !pip install nlpaug # For augmentation, if desired
# !pip install tf-keras

import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


/Users/stygianphantom/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
def clean_text(text):
    if not isinstance(text, str):  # Ensure text is a string
        return ""
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
df = pd.read_csv("cyberbullying_tweets.csv")

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['cyberbullying_type'])
label_encoder = LabelEncoder()
label_encoder.fit(df['cyberbullying_type'])

train_df['label'] = label_encoder.transform(train_df['cyberbullying_type'])
val_df['label'] = label_encoder.transform(val_df['cyberbullying_type'])

num_labels = len(label_encoder.classes_)

In [6]:
model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
def tokenize_function(examples):
    return tokenizer(examples['tweet_text'], truncation=True, padding='max_length', max_length=128)

In [8]:
# !pip install datasets
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map: 100%|██████████| 9539/9539 [00:00<00:00, 30125.14 examples/s]


In [9]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# !pip install --upgrade transformers accelerate torch

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True
)


/Users/stygianphantom/Library/Python/3.9/lib/python/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [12]:
trainer.train()

metrics = trainer.evaluate()
print("Validation metrics:", metrics)

model_save_path = "./trained_bert_cyberbullying"

# Save the trained model
trainer.save_model(model_save_path)

# Save the tokenizer as well
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.347700,0.332283,0.870217,0.867179,0.870867,0.870217
2,0.260100,0.366168,0.868959,0.865412,0.870484,0.868959
3,0.189800,0.403820,0.869274,0.867914,0.870206,0.869274


Validation metrics: {'eval_loss': 0.33228254318237305, 'eval_accuracy': 0.8702170038788133, 'eval_f1': 0.8671785137288573, 'eval_precision': 0.8708673777004419, 'eval_recall': 0.8702170038788133, 'eval_runtime': 93.6132, 'eval_samples_per_second': 101.898, 'eval_steps_per_second': 6.377, 'epoch': 3.0}
Model and tokenizer saved to ./trained_bert_cyberbullying


## Dataset 2 (Mendeley Data)

In [42]:
dfaggressive = pd.read_csv("3. Aggressive_All.csv")
dfnonaggressive = pd.read_csv("4. Non_Aggressive_All.csv")
dfaggressive

,No.,Message
0,1,zhha Islam does nothing but freeze the status ...
1,2,You dont get out much do you
2,3,MaxBlumenthal Campagnebds Blumenthal self prom...
3,4,No silly it isnt ITS UR MOMS and might I say q...
4,5,Yes there is even more rape in Muslim countrie...
...,...,...
118823,118824,I really hope Kat gets hit by a bus than rever...
118824,118825,Smarojit Chakravarty I know your Facebook prof...
118825,118826,Is it me or are you a fucking nb Have a nic...
118826,118827,This ARTICLE sucks major cock you people are p...


In [49]:
model = AutoModelForSequenceClassification.from_pretrained(model_save_path)
tokenizer = AutoTokenizer.from_pretrained(model_save_path)

In [43]:
from datasets import Dataset, concatenate_datasets

dfaggressive = dfaggressive[['Message']].rename(columns={'Message': 'tweet_text'})
dfnonaggressive = dfnonaggressive[['Message']].rename(columns={'Message': 'tweet_text'})

# 2️⃣ Add 'cyberbullying_type' column
dfaggressive['cyberbullying_type'] = 'cyberbullying'
dfnonaggressive['cyberbullying_type'] = 'not_cyberbullying'

# 3️⃣ Drop the unwanted column (e.g., 'No.' if it exists)
dfaggressive = dfaggressive.drop(columns=['No.'], errors='ignore')  # Ignore if column doesn't exist
dfnonaggressive = dfnonaggressive.drop(columns=['No.'], errors='ignore')

# 4️⃣ Combine the DataFrames
df_combined_mendeley = pd.concat([dfaggressive, dfnonaggressive], ignore_index=True)


In [47]:
df_combined_mendeley['tweet_text'] = df_combined_mendeley['tweet_text'].apply(clean_text)

train_mendeley_df, val__mendeley_df = train_test_split(df_combined_mendeley, test_size=0.2, random_state=42, stratify=df_combined_mendeley['cyberbullying_type'])
label_encoder = LabelEncoder()
label_encoder.fit(df_combined_mendeley['cyberbullying_type'])

train_mendeley_df['label'] = label_encoder.transform(train_mendeley_df['cyberbullying_type'])
val__mendeley_df['label'] = label_encoder.transform(val__mendeley_df['cyberbullying_type'])

In [48]:
train_dataset_mendeley = Dataset.from_pandas(train_mendeley_df)
val_dataset_mendeley = Dataset.from_pandas(val__mendeley_df)

train_dataset_mendeley = train_dataset_mendeley.map(tokenize_function, batched=True)
val_dataset_mendeley = val_dataset_mendeley.map(tokenize_function, batched=True)

train_dataset_mendeley.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset_mendeley.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map: 100%|██████████| 47532/47532 [00:02<00:00, 19163.77 examples/s]


In [50]:
trainer = Trainer(
    model=model,  
    args=training_args,  
    train_dataset=train_dataset_mendeley,  
    eval_dataset=val_dataset_mendeley, 
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.231800,0.247218,0.923020,0.923003,0.923410,0.923020
2,0.160200,0.216969,0.941429,0.941418,0.941759,0.941429
3,0.116600,0.233151,0.945552,0.945534,0.946167,0.945552


TrainOutput(global_step=35649, training_loss=0.1720109055589274, metrics={'train_runtime': 19896.6802, 'train_samples_per_second': 28.667, 'train_steps_per_second': 1.792, 'total_flos': 3.751914214550938e+16, 'train_loss': 0.1720109055589274, 'epoch': 3.0})

In [ ]:
model_save_path_mendeley = "./trained_bert_cyberbullying_mendeley"

# Save the trained model
trainer.save_model(model_save_path)

# Save the tokenizer as well
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")

Model and tokenizer saved to ./trained_bert_cyberbullying_mendeley


# Dataset 3 (CyberBullying Dataset)

In [64]:
# test
df_cbd = pd.read_csv("aggression_parsed_dataset.csv")
df_cbd = df_cbd.rename(columns={'Text': 'tweet_text'})
df_cbd = df_cbd.rename(columns={'oh_label': 'cyberbullying_type'})
df_cbd['tweet_text'] = df_cbd['tweet_text'].apply(clean_text)
label_mapping = {0: "not_cyberbullying", 1: "cyberbullying"}
df_cbd["cyberbullying_type"] = df_cbd["cyberbullying_type"].map(label_mapping)
df_cbd['label'] = label_encoder.transform(df_cbd['cyberbullying_type'])

print(df_cbd[['cyberbullying_type']].head())



  cyberbullying_type
0  not_cyberbullying
1  not_cyberbullying
2  not_cyberbullying
3  not_cyberbullying
4  not_cyberbullying


In [65]:
test_dataset = Dataset.from_pandas(df_cbd)
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
print("Test dataset is ready for evaluation!")

Map: 100%|██████████| 115864/115864 [00:07<00:00, 15969.25 examples/s]

Test dataset is ready for evaluation!


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_save_path_mendeley)

# Define the trainer
trainer = Trainer(
    model=model,
    eval_dataset=test_dataset,  # Test dataset
    compute_metrics=compute_metrics
)

# Evaluate the model
results = trainer.evaluate()

In [ ]:
print("Test Accuracy on the new dataset:", results)

Test Accuracy on the new dataset: {'eval_loss': 0.17330503463745117, 'eval_model_preparation_time': 0.0012, 'eval_accuracy': 0.9492163225850998, 'eval_f1': 0.9520162989756075, 'eval_precision': 0.9595308174932375, 'eval_recall': 0.9492163225850998, 'eval_runtime': 1135.7125, 'eval_samples_per_second': 102.019, 'eval_steps_per_second': 12.752}


In [83]:
trainer = Trainer(
    model=model
)

# Get raw model predictions
predictions = trainer.predict(test_dataset)
logits = predictions.predictions  # Raw model outputs
labels = predictions.label_ids  # Ground truth labels

# Convert logits to predictions (using softmax + threshold adjustment)
probs = np.exp(logits) / np.exp(logits).sum(axis=-1, keepdims=True)  # Softmax
threshold = 0.2  # Adjust this to increase recall
preds = (probs[:, 1] > threshold).astype(int)  # Classify cyberbullying cases

# Compute Evaluation Metrics
accuracy = accuracy_score(labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")

print(f"Results with threshold {threshold}:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")  # This should improve!
print(f"F1-score: {f1:.4f}")

Results with threshold 0.2:
Accuracy: 0.9517
Precision: 0.9604
Recall: 0.9517
F1-score: 0.9541


In [112]:
import torch
device = torch.device("cpu")  # Ensures it runs on CPU
model.to(device)

# Define test sentence
test_sentence = "This is garbage. You should quit"

# Tokenize and prepare input, ensuring tensor is also on CPU
inputs = tokenizer(test_sentence, return_tensors="pt", truncation=True, padding=True, max_length=128)
inputs = {key: value.to(device) for key, value in inputs.items()}  # Move input tensors to CPU

# Run inference
with torch.no_grad():
    outputs = model(**inputs)

# Get the prediction
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()

# Map predicted class to label
label_map = {1: "Not Cyberbullying", 0: "Cyberbullying"}
result = label_map[predicted_class]

print(f"Prediction: {result}")

Prediction: Cyberbullying


In [146]:
# Interpretability

import torch
from captum.attr import IntegratedGradients, LayerIntegratedGradients

# Ensure the model is on CPU
device = torch.device("cpu")
model.to(device)

# Define test sentence
test_sentence = "You do this one more time, I will kill you."

# Tokenize and prepare input (tensors on CPU)
inputs = tokenizer(test_sentence, return_tensors="pt", truncation=True, padding=True, max_length=128)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Run inference using input_ids (letting the model compute embeddings)
with torch.no_grad():
    outputs = model(**inputs)

# Get the prediction
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()

# Map predicted class to label
label_map = {1: "Not Cyberbullying", 0: "Cyberbullying"}
result = label_map[predicted_class]
print(f"Prediction: {result}")

# Define a forward function that takes token IDs and attention mask.
def forward_func(input_ids, attention_mask):
    output = model(input_ids=input_ids, attention_mask=attention_mask)
    # Return the logit for the predicted class
    return output.logits[:, predicted_class]

# Get the model's embedding layer. It will now be used because we pass input_ids.
embedding_layer = model.get_input_embeddings()

# Initialize Layer Integrated Gradients with the forward function and the embedding layer.
lig = LayerIntegratedGradients(forward_func, embedding_layer)

# Create a baseline using the pad token ID for input_ids.
baseline_input_ids = torch.full_like(inputs["input_ids"], tokenizer.pad_token_id).to(device)

# Compute attributions.
attributions, delta = lig.attribute(
    inputs=(inputs["input_ids"], inputs["attention_mask"]),
    baselines=(baseline_input_ids, inputs["attention_mask"]),
    return_convergence_delta=True
)

# Sum the attributions across the embedding dimension to get a single score per token.
attributions_sum = attributions.sum(dim=-1).squeeze(0)

# Convert token IDs back to tokens for visualization.
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())

print("\nToken Attributions (Layer Integrated Gradients):")
for token, score in zip(tokens, attributions_sum.detach().cpu().numpy()):
    print(f"{token}: {score:.4f}")


Prediction: Cyberbullying

Token Attributions (Layer Integrated Gradients):
[CLS]: -0.0697
you: 0.4282
do: -0.9938
this: 1.1640
one: 0.3671
more: 0.6768
time: 0.1341
,: 0.1318
i: 1.0055
will: 0.7858
kill: 2.6676
you: 1.6921
.: 0.8760
[SEP]: 3.4183
